# Extraindo termos mais frequentes

Este notebook processa dados textuais da CPI da Pandemia e extrai os termos mais relevantes de toda a base e os termos mais relevantes de cada sessão. Para isso, foi utilizado o conceito de TF-IDF. 

In [1]:
# Imports Necessários

import nltk 
import pandas as pd
import json 
from nltk import ngrams
import numpy as np 
from nltk.corpus import stopwords

# Stopwords em Português
stopwords_ptbr = stopwords.words('portuguese')

## Os Dados

Os dados foram extraídos da base de transcrições das sessões da pandemia que estão disponíveis no site da Base dos Dados: https://basedosdados.org/dataset/br-senado-cpipandemia. 

A seguinte consulta SQL foi utilizada: 

```sql
SELECT 
  data_sessao, 
  SUM(duracao_discurso) duracao_discursos, 
  STRING_AGG(texto_discurso, ' ') as discurso 
FROM 
  `basedosdados.br_senado_cpipandemia.discursos`
GROUP BY data_sessao
ORDER BY data_sessao
``` 

O resultado foi salvo no arquivo `input/discursos_por_dia.csv`.

In [3]:
all_data = pd.read_csv("../input/discursos_por_dia.csv")
all_data.head()

,data_sessao,duracao_discursos,discurso
0,2021-04-27,15120,"Invocando a proteção de Deus, declaro aberta a..."
1,2021-04-29,7920,"Havendo número regimental, declaro aberta a 2ª..."
2,2021-05-04,29040,"Bom dia. Havendo número regimental, declaro ab..."
3,2021-05-05,22560,"Bom dia. Havendo número regimental, declaro ab..."
4,2021-05-06,37320,"Bom dia! Havendo número regimental, declaro ab..."


Os dados foram tokenizados utilizando o `word_tokenize` do pacote NLTK mantendo apenas as palavras e retirando as pontuações. Para cada seção são extraídos os termos mais frequentes. Eles foram ordenados do termo mais frequente ao menos frequente.

In [5]:
corpus = {}
vocabulary = {}
for index, row in all_data.iterrows():
    print("Processing (%i/%i) ... %s" % (index + 1, all_data.shape[0], row['data_sessao'])) 
    content = [word.lower() for word in nltk.word_tokenize(row['discurso']) if word.isalpha() and word not in stopwords_ptbr]
    
    freq = nltk.FreqDist(content)

    corpus[row['data_sessao']] = dict(freq)

for day in corpus:
    corpus[day] = sorted(corpus[day].items(), key=lambda x: x[1], reverse=True)

Processing (1/38) ... 2021-04-27
Processing (2/38) ... 2021-04-29
Processing (3/38) ... 2021-05-04
Processing (4/38) ... 2021-05-05
Processing (5/38) ... 2021-05-06
Processing (6/38) ... 2021-05-11
Processing (7/38) ... 2021-05-12
Processing (8/38) ... 2021-05-13
Processing (9/38) ... 2021-05-18
Processing (10/38) ... 2021-05-19
Processing (11/38) ... 2021-05-20
Processing (12/38) ... 2021-05-25
Processing (13/38) ... 2021-05-26
Processing (14/38) ... 2021-05-27
Processing (15/38) ... 2021-06-01
Processing (16/38) ... 2021-06-02
Processing (17/38) ... 2021-06-08
Processing (18/38) ... 2021-06-09
Processing (19/38) ... 2021-06-10
Processing (20/38) ... 2021-06-11
Processing (21/38) ... 2021-06-15
Processing (22/38) ... 2021-06-16
Processing (23/38) ... 2021-06-17
Processing (24/38) ... 2021-06-18
Processing (25/38) ... 2021-06-22
Processing (26/38) ... 2021-06-23
Processing (27/38) ... 2021-06-24
Processing (28/38) ... 2021-06-25
Processing (29/38) ... 2021-06-29
Processing (30/38) ... 

Um vocabulário das palavras de todos os textos também criado. Nele são armazenado: o total de documentos que o texto aparece, a freqência do termo na coleção e a lista de documentos na qual o termo aparece. Isso será utilizado para o cálculo do TF-IDF de cada termo/documento.

In [6]:
vocabulary = {}
total_words = 0
for day in corpus:
    print("Processing ... %s" % day) 
    for word, score in corpus[day]:
        total_words += 1
        if word not in vocabulary.keys():
            vocabulary[word] = {'total': 1, 'documents': [day], 'freq': score}
        else:
            vocabulary[word]['total'] += 1
            vocabulary[word]['freq'] += score 
            vocabulary[word]['documents'].append(day)
        

Processing ... 2021-04-27
Processing ... 2021-04-29
Processing ... 2021-05-04
Processing ... 2021-05-05
Processing ... 2021-05-06
Processing ... 2021-05-11
Processing ... 2021-05-12
Processing ... 2021-05-13
Processing ... 2021-05-18
Processing ... 2021-05-19
Processing ... 2021-05-20
Processing ... 2021-05-25
Processing ... 2021-05-26
Processing ... 2021-05-27
Processing ... 2021-06-01
Processing ... 2021-06-02
Processing ... 2021-06-08
Processing ... 2021-06-09
Processing ... 2021-06-10
Processing ... 2021-06-11
Processing ... 2021-06-15
Processing ... 2021-06-16
Processing ... 2021-06-17
Processing ... 2021-06-18
Processing ... 2021-06-22
Processing ... 2021-06-23
Processing ... 2021-06-24
Processing ... 2021-06-25
Processing ... 2021-06-29
Processing ... 2021-06-30
Processing ... 2021-07-01
Processing ... 2021-07-06
Processing ... 2021-07-07
Processing ... 2021-07-08
Processing ... 2021-07-09
Processing ... 2021-07-13
Processing ... 2021-07-14
Processing ... 2021-07-15


## TF-IDF para cada documento

Para cada par termo/documento é calculado o TF-IDF. Foi utilizada a seguinte fórmula: 

$TFIDF(t, d) = TF * IDF$

onde,

$TF = \frac{f_t}{|d|}$

onde, $f$ é a frequência do termo no documento e $d$ o total de palavras no documento. 

$IDF = log_{10}(\frac{N}{Total_{d,t}})$

onde $N$ é o total de documentos da base (cada dia foi considerado um documento) e $Total_{d,t}$ é  o número de documentos que o termo aparece. 

O trecho de código a seguir faz o cálculo.

In [7]:
final_corpus = {}
short_corpus = {}
total_documents = len(corpus.keys())
acc_total_words = 0
for day in corpus:
    final_corpus[day] = {}
    content = corpus[day]
    total_words = len(set(content))
    acc_total_words += total_words
    dict_ = {
        word[0]: {
            'freq': word[1], 
            'tf-idf': (word[1] / total_words) * np.log10(total_documents / vocabulary[word[0]]['total']), 
            'documents': vocabulary[word[0]]['documents']
            } for word in content}
    final_corpus[day] = dict_
    final_corpus[day] = sorted(final_corpus[day].items(), key=lambda x: x[1]['tf-idf'], reverse=True)
    short_corpus[day] = dict(final_corpus[day][:250])
    

O código acima foi utilizado para gerar os termos mais relevantes de cada dia de sessão da CPI. No entanto, eu queria extrair informação parecida de toda a coleção. Para isso, foi utilizado o IDF de cada termo ponderado pela frequência dele em toda coleção. O código a seguir faz este cálculo. 

In [9]:
for voc in vocabulary:
    vocabulary[voc]['idf'] = (vocabulary[voc]['freq']) * (np.log10(len(corpus.keys()) / vocabulary[voc]['total']))

vocab_sorted = sorted(vocabulary.items(), key=lambda x: x[1]['idf'], reverse=True)


Com isso foram gerados dois arquivos de saída:

* `corpus_tfid.json`: com o TF-IDF de cada termo/documento. Observe que salvamos apenas os 250 termos mais relevantes de cada dia. Por isso, estamos salvando a variável `short_corpus`. 
* `vocabulary_full.json`: com o IDF ponderado pela frequência de toda a base de dados.

In [10]:
with open("../output/corpus_tfid.json", 'w') as outfile:
    json.dump(short_corpus, outfile)

In [11]:
with open("../output/vocabulary_full.json", 'w') as outfile:
    json.dump(vocab_sorted, outfile)